<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Eduardo Moya
- Nombre de alumno 2: Nicolás Ojeda


### **Link de repositorio de GitHub:** `https://github.com/eduardomoyab/MDS7202_1`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
#Cargamos el archivo pickle en la misma carpeta llamado online_retail_II_cleaned.pickle
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [5]:
import pandas as pd

def custom_features(dataframe_in):
    # Crear una copia del DataFrame para no modificar el original
    df = dataframe_in.copy()

    # Convertir la columna 'InvoiceDate' a tipo de datos datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

    # Calcular la característica Length (L)
    length_df = df.groupby('Customer ID')['InvoiceDate'].agg(lambda x: (x.max() - x.min()).days).rename('Length')

    # Calcular la característica Recency (R)
    recency_df = df.groupby('Customer ID')['InvoiceDate'].max()
    recency_df = (df['InvoiceDate'].max() - recency_df).dt.days.rename('Recency')

    # Calcular la característica Monetary (M)
    df['TotalPrice'] = df['Quantity'] * df['Price']
    monetary_df = df.groupby('Customer ID')['TotalPrice'].mean().rename('Monetary')

    # Calcular la característica Frequency (F) eliminando duplicados
    frequency_df = df.drop_duplicates(subset=['InvoiceDate', 'Customer ID']).groupby('Customer ID').size().rename('Frequency')

    # Calcular la característica Periodicity (P)
    ivt_df = df.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    periodicity_df = ivt_df.groupby(df['Customer ID']).std().rename('Periodicity')

    # Crear un nuevo DataFrame con las características calculadas
    result_df = pd.concat([length_df, recency_df, frequency_df, monetary_df, periodicity_df], axis=1)

    return result_df

# Llamar a la función y guardar el resultado en un nuevo DataFrame
result = custom_features(df_retail)
result.head()


,Length,Recency,Frequency,Monetary,Periodicity
Customer ID,,,,,
12346.0,196,164,11,11.298788,21.724076
12347.0,37,2,2,18.638310,4.422346
12348.0,0,73,1,11.108000,0.000000
12349.0,181,42,3,26.187647,16.200990
12351.0,0,10,1,14.330000,0.000000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [6]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self, X):
        self.min_vals = X.min()
        self.max_vals = X.max()
        return self

    def transform(self, X):
        # Aplicar la transformación Min-Max a cada columna
        X_transformed = (X - self.min_vals) / (self.max_vals - self.min_vals)

        return X_transformed

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [19]:
#Se usa FunctionTransformer para convertir la función definida anteriormente en un transformador
feature_extractor = FunctionTransformer(custom_features, validate=False)

# Como la funcion custom_features puede generar NaN en "Periodicity", imputamos los datos, esto con simple imputer
imputer = SimpleImputer(strategy='mean')

# Seleccionamos todas las columnas obtenidas de custom_features y la escalamos
transformer = ColumnTransformer(
    transformers=[
        ('minmax', 
         MinMax(), 
         slice(0, None)) 
    ],
    remainder='passthrough'
)

#realizamos la reduccion de dimensionalidad a dos componentes
tsne = TSNE(n_components=2)

# creamos el pipeline para el flujo de trabajo
pipeline = Pipeline([
    ('feature_extraction', feature_extractor),
    ('imputation', imputer),  # Paso de imputación
    ('scaling', transformer),
    ('dimension_reduction', tsne)
])

# Ajustar el pipeline a los datos
reduced_features = pipeline.fit_transform(df_retail)

# Crear un DataFrame con las componentes reducidas
reduced_df = pd.DataFrame(data=reduced_features, columns=["Component 1", "Component 2"])

fig = px.scatter(
    reduced_df, x="Component 1", y="Component 2",
    title="Gráfico de Dispersión de Componentes Reducidas (T-SNE)",
    labels={'Component 1': 'TSNE Component 1', 'Component 2': 'TSNE Component 2'},
    width=800, height=600
)

fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Reaalice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [20]:
# Números de clusters a probar
k_values = range(1, 21)
inertia_values = []

for k in k_values:
    # Crear un nuevo pipeline con K-Means y el número de clusters actual
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_pipeline = Pipeline([
        ('features', feature_extractor),
        ('impute', imputer),
        ('scaler', transformer),
        ('clustering', kmeans)
    ])
    
    # Aplicar el pipeline y obtener la Inertia
    cluster_pipeline.fit_transform(df_retail)
    inertia = cluster_pipeline.named_steps['clustering'].inertia_
    inertia_values.append(inertia)

# Crear un DataFrame con los valores de k y la Inertia
elbow_data = pd.DataFrame({'k': k_values, 'Inertia': inertia_values})

# Crear un gráfico interactivo con Plotly
fig = px.line(elbow_data, x='k', y='Inertia', title='Método del Codo para Determinar el Valor Óptimo de k')
fig.update_traces(mode='lines+markers')
fig.update_xaxes(title='Número de Clusters (k)')
fig.update_yaxes(title='Inertia')
fig.show()

Basándonos en los resultados del método del codo, se opta por seleccionar un valor de k igual a 3, ya que a partir de este punto, la inercia no experimenta una disminución sustancial y disminuye de manera muy leve. Por lo tanto, se puede considerar que k = 3 es el valor óptimo, lo que sugiere que la segmentación en tres grupos es la más adecuada para representar de manera efectiva la estructura de los datos.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [21]:
# Número óptimo de clusters (elegido en la sección anterior)
optimal_k = 3  

# Creamos un nuevo K-Means con el número óptimo de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)

# Crear un nuevo pipeline con K-Means
cluster_pipeline = Pipeline([
    ('features', feature_extractor),
    ('impute', imputer),
    ('scaler', transformer),
    ('clustering', kmeans)
])

# Ajustar el modelo a los datos
cluster_pipeline.fit_transform(df_retail)

# Obtener las etiquetas de clústeres
labels = cluster_pipeline.named_steps['clustering'].labels_

# Crear una copia del DataFrame original para evitar errores de longitud
df_retail_copy = custom_features(df_retail)

# Aplicar el imputador al DataFrame copiado
imputed_df = imputer.fit_transform(df_retail_copy)

# Convertir el resultado en un DataFrame
imputed_df = pd.DataFrame(imputed_df, columns=df_retail_copy.columns)

# Agregar las etiquetas de clústeres al DataFrame copiado
imputed_df['Cluster'] = labels

# Calcular los promedios para cada atributo agrupados por clúster
cluster_means = imputed_df.groupby('Cluster').agg({
    'Length': 'mean',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'Periodicity': 'mean',
})

# Mostrar la tabla de promedios
cluster_means


,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,
0,269.766206,35.450584,7.841658,31.821931,20.693854
1,28.848560,132.583128,1.818107,33.646554,4.902649
2,0.000000,263.000000,1.500000,8955.835000,11.794821


**Cluster 0:**
En el primer cluster, se ha identificado un grupo de clientes que presenta características coherentes y sugiere una alta fidelidad a lo largo del tiempo. Estos clientes han mantenido un intervalo de tiempo considerable entre su primera y última visita, con una longitud promedio de 269.77 días. Además, su recencia, es decir, el tiempo transcurrido desde su última compra, es relativamente bajo, con un promedio de 35.45 días. Esto indica que estos clientes tienden a regresar a los locales con frecuencia. Realizan un promedio de 7.84 visitas, contribuyendo a los ingresos de la empresa con un gasto promedio de $31.82 por visita. La baja variabilidad en sus patrones de compra, reflejada en una periodicidad de 20.69, respalda aún más su lealtad. Este cluster representa a un tipo de cliente que posee un alto potencial para la retención y fidelización, lo que es esencial para el éxito del negocio.

**Cluster 1:**
El segundo cluster muestra un grupo de clientes que, aunque presenta una fidelidad moderada, ha extendido el tiempo desde su última compra, con una recencia promedio de 132.58 días. A pesar de esta menor frecuencia de visitas (1.82 visitas en promedio), estos clientes mantienen un gasto promedio por visita similar al Cluster 0, con $33.65. La baja variabilidad en sus patrones de compra, indicada por una periodicidad de 4.90, sugiere cierta consistencia en su comportamiento. Aunque han pasado más tiempo desde su última compra, estos clientes aún tienen el potencial de ser retenidos y reactivados mediante estrategias de marketing específicas. Este cluster representa un tipo de cliente que puede beneficiarse de iniciativas de retención dirigidas a recuperar su interés en las compras.

**Cluster 2:**
El tercer cluster presenta características únicas en comparación con los dos anteriores. A pesar de no mostrar un intervalo de tiempo entre su primera y última visita (length de 0 días), estos clientes han pasado un tiempo considerable desde su última compra, con una recencia promedio de 263 días. Realizan 1.5 visitas en promedio, lo que podría sugerir un patrón de compra más esporádico. Sin embargo, lo más destacado de este grupo es el gasto promedio significativamente más alto por visita, con $8955.84. La baja variabilidad en sus patrones de compra, representada por una periodicidad de 11.79, indica cierta regularidad en su comportamiento de compra a pesar del largo período entre visitas. Este cluster podría representar un tipo de cliente que busca productos de alto valor y calidad en las tiendas, donde se podrían adaptar estrategias para mantener su satisfacción y retener su lealtad.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [22]:
import plotly.express as px

# Agregar las etiquetas de clústeres al DataFrame reduced_df
reduced_df['Cluster'] = labels

# Definir etiquetas personalizadas para los clusters
cluster_labels = {
    0: "Cluster 0",
    1: "Cluster 1",
    2: "Cluster 2"
    # Agrega etiquetas para otros clusters si es necesario
}

# Mapear las etiquetas personalizadas a los clusters
reduced_df['Cluster_Label'] = reduced_df['Cluster'].map(cluster_labels)

# Crear un gráfico de dispersión con etiquetas personalizadas
fig = px.scatter(
    reduced_df, x="Component 1", y="Component 2", color="Cluster_Label",
    title="Gráfico de Dispersión de Componentes Reducidas (T-SNE) con Etiquetas de Clústeres",
    labels={'Component 1': 'TSNE Component 1', 'Component 2': 'TSNE Component 2'},
    width=800, height=600
)

fig.show()


En el gráfico de dispersión que utiliza T-SNE para visualizar las componentes reducidas con las etiquetas de clústeres agregadas, se observa que los clusters no están perfectamente separados, lo que sugiere cierta superposición en las características de los clientes. La principal distinción entre los clusters se relaciona con los valores de T-SNE Component 2, donde la mayoría de los puntos con valores menores a cero corresponden al Cluster 1 y los valores mayores al Cluster 2, lo que refleja diferencias en la recencia de las compras. Además, se nota que hay un tipo de cliente único que muestra compras de alto valor y un patrón esporádico, representado por un punto casi solitario. Sin embargo, en esta representación bidimensional, se aprecian numerosas pequeñas agrupaciones, lo que podría indicar que el método K-Means puede no ser la elección óptima para este conjunto de datos. Explorar otras técnicas de clustering, como DBSCAN, podría proporcionar una perspectiva más detallada y refinada de las relaciones entre los clientes.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>